In [ ]:
!pip -q install langchain huggingface_hub tiktoken
!pip -q install chromadb
!pip -q install PyPDF2 pypdf sentence_transformers
!pip -q install --upgrade together

!pip -q install -U FlagEmbedding

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 82.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.5/405.5 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 83.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 112.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 55.0 MB/

## RetrievalQA with LLaMA 2-7B on Together API

In [ ]:
import os

os.environ["TOGETHER_API_KEY"] = ""

In [ ]:
!pip show langchain

Name: langchain
Version: 0.0.267
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, langsmith, numexpr, numpy, openapi-schema-pydantic, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


# Setting up Together API


In [ ]:
import together

# set your API key
together.api_key = os.environ["TOGETHER_API_KEY"]

# list available models and descriptons
models = together.Models.list()

In [ ]:
together.Models.start("togethercomputer/llama-2-7b-chat")

{'success': True,
 'value': 'cdb7f61afe17889c49a9a67ed33373ee74fb490214d4789b75e2ee4bae88dfd8'}

In [ ]:
import together

import logging
from typing import Any, Dict, List, Mapping, Optional

from pydantic import Extra, Field, root_validator

from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM
from langchain.llms.utils import enforce_stop_tokens
from langchain.utils import get_from_dict_or_env

class TogetherLLM(LLM):
    """Together large language models."""

    model: str = "togethercomputer/llama-2-70b-chat"
    """model endpoint to use"""

    together_api_key: str = os.environ["TOGETHER_API_KEY"]
    """Together API key"""

    temperature: float = 0.7
    """What sampling temperature to use."""

    max_tokens: int = 512
    """The maximum number of tokens to generate in the completion."""

    class Config:
        extra = Extra.forbid

    @root_validator()
    def validate_environment(cls, values: Dict) -> Dict:
        """Validate that the API key is set."""
        api_key = get_from_dict_or_env(
            values, "together_api_key", "TOGETHER_API_KEY"
        )
        values["together_api_key"] = api_key
        return values

    @property
    def _llm_type(self) -> str:
        """Return type of LLM."""
        return "together"

    def _call(
        self,
        prompt: str,
        **kwargs: Any,
    ) -> str:
        """Call to Together endpoint."""
        together.api_key = self.together_api_key
        output = together.Complete.create(prompt,
                                          model=self.model,
                                          max_tokens=self.max_tokens,
                                          temperature=self.temperature,
                                          )
        text = output['output']['choices'][0]['text']
        return text


In [ ]:

!wget -O new_papers_2.zip https://www.dropbox.com/scl/fi/67a80h373n1z38088c9fb/new_papers_2.zip?rlkey=1azfz3w5aazd24ihotwzmol2j&dl=1
!unzip -q new_papers_2.zip -d new_papers

--2023-08-17 22:47:53--  https://www.dropbox.com/scl/fi/67a80h373n1z38088c9fb/new_papers_2.zip?rlkey=1azfz3w5aazd24ihotwzmol2j
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6025:18::a27d:4512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/e/scl/fi/67a80h373n1z38088c9fb/new_papers_2.zip?rlkey=1azfz3w5aazd24ihotwzmol2j [following]
--2023-08-17 22:47:53--  https://www.dropbox.com/e/scl/fi/67a80h373n1z38088c9fb/new_papers_2.zip?rlkey=1azfz3w5aazd24ihotwzmol2j
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucce9f909eb5993eed67a699eda1.dl.dropboxusercontent.com/cd/0/get/CB-1kw9BxtsJCkpytEZzbYbIbV3N0VdNqewVIlSmfCaXNwFe_yDciETGrCRxElBz9PoSZ-VjTcpyYNFl_YMm5YUK73CXyxhx_fSb53PM3rGnZAeT9Z9UnEk_vEle2BUY1W0/file# [following]
--2023-08-17 22:47:54--  https://ucce9f909eb5993eed67a699eda1.d

# LangChain multi-doc retriever with ChromaDB

***Key Points***
- Multiple Files - PDFs
- ChromaDB
- LLaMA-2 LLM
- BGE Embeddings


## Setting up LangChain


In [ ]:
import os

In [ ]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader



## Load multiple and process documents

In [ ]:
# Load and process the text files
# loader = TextLoader('single_text_file.txt')
loader = DirectoryLoader('./new_papers/new_papers/', glob="./*.pdf", loader_cls=PyPDFLoader)

documents = loader.load()

In [ ]:
len(documents)

219

In [ ]:
#splitting the text into
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

len(texts)

992

In [ ]:
texts[3]

Document(page_content='Contents\n1 Introduction 3\n2 Pretraining 5\n2.1 Pretraining Data . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 5\n2.2 Training Details . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 5\n2.3 Llama 2 Pretrained Model Evaluation . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 7\n3 Fine-tuning 8\n3.1 Supervised Fine-Tuning (SFT) . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 9\n3.2 Reinforcement Learning with Human Feedback (RLHF) . . . . . . . . . . . . . . . . . . . . . 9\n3.3 System Message for Multi-Turn Consistency . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 16\n3.4 RLHF Results . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 17\n4 Safety 20\n4.1 Safety in Pretraining . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 20', metadata={'source': 'n

## HF BGE Embeddings

In [ ]:

from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-base-en"
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

model_norm = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cuda'},
    encode_kwargs=encode_kwargs
)


## create the DB

 T4 GPU

In [ ]:
%%time
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk

persist_directory = 'db'

## Here is the nmew embeddings being used
embedding = model_norm

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

CPU times: user 26.2 s, sys: 1.23 s, total: 27.4 s
Wall time: 32.8 s


## Make a retriever

In [ ]:
retriever = vectordb.as_retriever(search_kwargs={"k": 5})

## Make a chain

In [ ]:
## Default LLaMA-2 prompt style
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."""

def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT ):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST
    return prompt_template

In [ ]:
sys_prompt = """You are a helpful, respectful and honest assistant. Always answer as helpfully as possible using the context text provided. Your answers should only answer the question once and not have any text after the answer is done.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. """

instruction = """CONTEXT:/n/n {context}/n

Question: {question}"""
get_prompt(instruction, sys_prompt)

"[INST]<<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible using the context text provided. Your answers should only answer the question once and not have any text after the answer is done.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. \n<</SYS>>\n\nCONTEXT:/n/n {context}/n\n\nQuestion: {question}[/INST]"

In [ ]:
llm = TogetherLLM(
    model= "togethercomputer/llama-2-7b-chat",
    temperature = 0.1,
    max_tokens = 1024
)

In [ ]:
from langchain.prompts import PromptTemplate
prompt_template = get_prompt(instruction, sys_prompt)

llama_prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [ ]:
chain_type_kwargs = {"prompt": llama_prompt}


In [ ]:
from langchain.schema import prompt
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                       chain_type="stuff",
                                       retriever=retriever,
                                       chain_type_kwargs=chain_type_kwargs,
                                       return_source_documents=True)



In [ ]:
## Cite sources

import textwrap

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

def process_llm_response(llm_response):
    print(wrap_text_preserve_newlines(llm_response['result']))
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [ ]:
# full example
query = "What is Flash attention?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 FlashAttention is a new attention algorithm that computes exact attention with far fewer memory accesses than
traditional attention mechanisms. It achieves this by restructuring the attention computation to split the
input into blocks and make several passes over input blocks, thus incrementally performing the softmax
reduction. Additionally, FlashAttention stores the softmax normalization factor from the forward pass to
quickly recompute attention on-chip in the backward pass, which is faster than the standard approach of
reading the intermediate attention matrix from HBM.


Sources:
new_papers/new_papers/Flash-attention.pdf
new_papers/new_papers/Flash-attention.pdf
new_papers/new_papers/Flash-attention.pdf
new_papers/new_papers/Flash-attention.pdf
new_papers/new_papers/Flash-attention.pdf


In [ ]:
# break it down
query = "What does IO-aware mean?"
llm_response = qa_chain(query)
process_llm_response(llm_response)
# llm_response

 IO-aware refers to the ability of a deep learning model or algorithm to carefully account for and manage the
input/output (I/O) operations required for its computation. In the context of attention, IO-awareness means
considering the reads and writes to different levels of fast and slow memory (e.g., between GPU on-chip SRAM
and HBM) and optimizing the attention computation to minimize the number of I/O operations. The goal is to
reduce the overall memory bandwidth and improve the memory efficiency of the model.


Sources:
new_papers/new_papers/Flash-attention.pdf
new_papers/new_papers/Flash-attention.pdf
new_papers/new_papers/Flash-attention.pdf
new_papers/new_papers/Flash-attention.pdf
new_papers/new_papers/Flash-attention.pdf


In [ ]:
query = "What is the context window for LLaMA-2?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 The context window for LLaMA-2 is 4096 tokens.


Sources:
new_papers/new_papers/llama-2-paper.pdf
new_papers/new_papers/llama-2-paper.pdf
new_papers/new_papers/llama-2-paper.pdf
new_papers/new_papers/llama-2-paper.pdf
new_papers/new_papers/llama-2-paper.pdf


In [ ]:
query = "How many tokens was LLaMA-2 trained on?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 The LLaMA-2 model was trained on 2 trillion tokens.


Sources:
new_papers/new_papers/llama-2-paper.pdf
new_papers/new_papers/llama-2-paper.pdf
new_papers/new_papers/llama-2-paper.pdf
new_papers/new_papers/llama-2-paper.pdf
new_papers/new_papers/llama-2-paper.pdf


In [ ]:
query = "When is LLaMA-3 coming?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 The answer to your question is not available as the text you provided does not mention any information about
the release date of LLaMA-3. The text only provides information about the current version of LLaMA, which is
LLaMA-2, and its performance on various benchmarks.


Sources:
new_papers/new_papers/llama-2-paper.pdf
new_papers/new_papers/llama-2-paper.pdf
new_papers/new_papers/llama-2-paper.pdf
new_papers/new_papers/llama-2-paper.pdf
new_papers/new_papers/llama-2-paper.pdf


In [ ]:
query = "What is the new model from Meta called?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 The new model from Meta is called Llama 2.


Sources:
new_papers/new_papers/llama-2-paper.pdf
new_papers/new_papers/llama-2-paper.pdf
new_papers/new_papers/llama-2-paper.pdf
new_papers/new_papers/llama-2-paper.pdf
new_papers/new_papers/llama-2-paper.pdf


In [ ]:
query = "What is toolformer?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Toolformer is a language model that has been trained to use different tools, such as search engines,
calculators, and translation systems, via simple API calls. It is based on a pre-trained GPT-J model with 6.7
billion parameters and achieves much stronger zero-shot results than a much larger GPT-3 model on various
tasks. Toolformer can be used to equip a language model with the ability to use different tools by means of
API calls, and it represents each API call as a tuple (ac, ic) where ac is the name of the API and ic is the
corresponding input.


Sources:
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/toolformer.pdf


In [ ]:
query = "What tools can be used with toolformer?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Toolformer can be used with various tools, but the specific tools it can use depend on the task and the
dataset used for training. In the context of the paper, Toolformer is trained on a dataset of math problems
and uses the calculator tool for most examples. However, the approach does not limit the use of tools to a
specific set and can potentially be applied to other tools as well.


Sources:
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/llama-2-paper.pdf
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/toolformer.pdf
new_papers/new_papers/llama-2-paper.pdf


In [ ]:
query = "How many examples do we need to provide for each tool?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 The number of examples required for each tool depends on various factors, including the complexity of the
tool, the type of task it can perform, and the amount of training data available. In general, more examples
are needed for more complex tools or tasks, and for tools that require a higher level of understanding and
reasoning.

There is no fixed number of examples that can be applied to all tools, as each tool has its unique
characteristics and requirements. However, as a general guideline, we can provide some rough estimates of the
number of examples that may be needed for different types of tools:

* For simple tools with a limited number of functions, such as a document retriever or a calculator, a few
hundred examples may be sufficient.
* For more complex tools with multiple functions or sub-tasks, such as a web search engine or a language
translator, thousands of examples may be required to achieve good performance.
* For tools that require a high level of understanding and re

In [ ]:
query = "What are the best retrieval augmentations for LLMs?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 The best retrieval augmentations for LLMs depend on the specific task and context. However, some popular and
effective retrieval augmentations for LLMs include:

1. Dense and sparse retrievers: These types of retrievers can be used to augment a LM by retrieving documents
based on a dense or sparse query. Dense retrievers use a dense query to retrieve documents, while sparse
retrievers use a sparse bag-of-words representation of the queries.
2. Memory units: Augmenting a LM with memory units, such as a neuronal cache of recent inputs, can improve its
reasoning abilities and reduce the number of parameters required to achieve comparable performance to larger
LMs.
3. External knowledge sources: Retrieving knowledge from external sources, such as a knowledge graph or a
database, can provide a LM with additional information to inform its predictions.
4. Task-oriented supervised data: Using task-oriented supervised data to train a LM can help it learn to
predict the next token in a sequence

In [ ]:
query = "What is ReAct?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 ReAct is a language model that is trained to perform various tasks, including question answering, fact
verification, and web navigation. It is designed to be helpful, respectful, and honest, and it only answers
questions once without providing unnecessary information. ReAct is able to perform these tasks by interleaving
reasoning and acting, allowing for greater synergy between the two and improved performance on both language
and decision-making tasks. It has been deployed on a public website to continually learn and improve with
humans in the loop, and it uses few-shot prompting to teach a language model how to use different tools such
as search and lookup in Wikipedia. ReAct performs well on a diverse set of language and decision-making tasks,
including question answering, fact verification, and web and home navigation.


Sources:
new_papers/new_papers/ReACT.pdf
new_papers/new_papers/ReACT.pdf
new_papers/new_papers/ReACT.pdf
new_papers/new_papers/Augmenting LLMs Survey.pdf
new_pape

In [ ]:
qa_chain.retriever.search_type , qa_chain.retriever.vectorstore

('similarity', <langchain.vectorstores.chroma.Chroma at 0x7e49506ea410>)

In [ ]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.template)

[INST]<<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible using the context text provided. Your answers should only answer the question once and not have any text after the answer is done.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. 
<</SYS>>

CONTEXT:/n/n {context}/n

Question: {question}[/INST]


In [ ]:
together.Models.stop("togethercomputer/llama-2-7b-chat")

{'success': True}